giusto per capirci, quello che poi va usato nel notebook ufficiale è quello contenuto in text_enrichment.py
che fa tutto ilnecessario

angolo del text enrichment, qua non si babbia

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import sys
import re

wd = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(wd)

random_state=69



In [2]:
# from utils import set_seeds, preprocessing

In [3]:
cols = ['iro', 'text']

df1 = pd.read_csv(wd + "/data/training_set_sentipolc16.csv", sep=",")

# stesse colonne, lultima colonna è il topic (non è binaria)
df2 = pd.read_csv(wd + "/data/test_set_sentipolc16_gold2000.csv", sep=",",names=list(df1.columns))

df1 = pd.concat([df1, df2])[cols]
df = df1.copy()
df

,iro,text
0,0,Intanto la partita per Via Nazionale si compli...
1,0,"False illusioni, sgradevoli realtà Mario Monti..."
2,0,"False illusioni, sgradevoli realtà #editoriale..."
3,0,Mario Monti: Berlusconi risparmi all'Italia il...
4,0,Mario Monti: Berlusconi risparmi all'Italia il...
...,...,...
1995,0,Anche prodotti alimentari tipici pugliesi in v...
1996,0,intensità di vita https://t.co/jv4aARxzhz
1997,0,Oggi tutti che iniziano l'università e io sul ...
1998,0,@GliIntoccabili @nonleggerlo Ma Ferrero? il co...


In [4]:
# df[df['text'].apply(lambda x : 'monti?!' in x.lower())]

In [5]:
from text_enrichment import create_occurrences_dict

df['list'] = df['text'].apply(lambda x : x.split())

dizionario = create_occurrences_dict(df, colname='list')

# chiavi = set(dizionario[1].keys()).intersection(set(dizionario[0].keys()))

# differenze = {
#     key: (dizionario[1][key], dizionario[0][key], dizionario[1][key] - dizionario[0][key])
#     for key in chiavi
# }

# dizionario_ordinato = dict(sorted(differenze.items(), key=lambda x: x[1][2], reverse=True))

# for key, (val1, val2, diff) in dizionario_ordinato.items():
#     print(f'{key}\t\ttrue {val1}, false {val2}, differenza {diff}')

In [6]:
# dizionario_ordinato

In [7]:
# parole_false_ordinato = sorted(dizionario[0].items(), key=lambda x: x[1], reverse=True)
# parole_vere_ordinato = sorted(dizionario[0].items(), key=lambda x: x[1], reverse=True)

In [8]:
# from text_enrichment import get_prob_from_sentence

# get_prob_from_sentence('suca #mannaggi haha', features_vals)

In [9]:
# train_set, val_set = train_test_split(df,
#                                      test_size=0.3,
#                                      shuffle=True,
#                                      random_state=random_state
#                     )
train_set = df
train_set.shape

(9410, 3)

In [10]:
# val_set.shape

In [11]:
def estrai_hastag(testo): return [hashtag.lstrip('#') for hashtag in re.findall(r'#\w+', testo)] 

In [12]:
# i link non contengono nessuna informazione.
# in modo che vengano trattati come hastag 

link_as_hastag = False
if link_as_hastag:
    train_set['text'] = train_set['text'].apply(lambda x: re.sub(r'https?v?:\/\/\S+', '#LINK', x))
else:
    train_set['text'] = train_set['text'].apply(lambda x: re.sub(r'https?v?:\/\/\S+', '', x))
train_set['text'] = train_set['text'].apply(lambda x: re.sub(r'\b(?:ha|he|hi|ho|ah|eh|ih|oh|uh){2,}\b', '#RISATA', x))
train_set['hastags'] = train_set['text'].apply(estrai_hastag)

levo tutti i tweet che non contengono info utili (hastag, risate, link )<br>
training_set_sentipolc16.csv<br>
&emsp; 1700(non iro) + 315(iro) =  2015 (tweet senza hastag)<br>
l'altro dataset mi sembra inutilizzabile idk

In [13]:
tmp_df_hasth = train_set[['iro','hastags']].copy()
tmp_df_hasth = tmp_df_hasth[tmp_df_hasth['hastags'].apply(lambda x: len(x) > 0)]
tmp_df_hasth

,iro,hastags
0,0,"[Saccomanni, Monti]"
2,0,"[editoriale, rassegna]"
3,0,[mariomontipremier]
14,0,[ottoemezzo]
15,1,[la7]
...,...,...
1979,0,[modenafc1912]
1984,0,"[Piacenza, OroRosso]"
1989,0,[WeWillMeetAgain1D]
1990,0,[39]


<center>
    <h5>QUESTO CODICE NON TIENE CONTO DELLE CO-OCCORRENZE</h5>
</center>

In [14]:
tmd_dict = {0: {}, 1:{}}
for iro, elems  in tmp_df_hasth.values:
    for elem in elems:
        if elem.lower() in tmd_dict[iro]:
            tmd_dict[iro][elem.lower()] += 1
        else:
            tmd_dict[iro][elem.lower()] = 1


In [15]:
# sorted_hashtags = sorted(tmd_dict[0].items(), key=lambda x: x[1], reverse=True)
tot_n_iro = sum(tmd_dict[0].values()) # somma di tutti gli hastag contenuti in tweets non ironici
n_top_tweet = 15
prob_thr = 0.01

relevant_tweets = set()
p_hastg_non_iro = {}

for _, elem in enumerate(sorted(tmd_dict[0].items(), key=lambda x: x[1], reverse=True)):
    if (elem[1]/tot_n_iro) < prob_thr : break
    if _ > n_top_tweet : break
    
    hashtag = elem[0].ljust(20)  # Allinea l'hashtag a sinistra con 20 caratteri di spazio
    print(f'P({hashtag}|iro=0)\t= {elem[1]/tot_n_iro:.4f}')

    relevant_tweets.add(elem[0])
    p_hastg_non_iro[elem[0]] = elem[1]/tot_n_iro


P(grillo              |iro=0)	= 0.1366
P(labuonascuola       |iro=0)	= 0.1262
P(monti               |iro=0)	= 0.0945
P(governo             |iro=0)	= 0.0276
P(serviziopubblico    |iro=0)	= 0.0172
P(piazzapulita        |iro=0)	= 0.0131
P(m5s                 |iro=0)	= 0.0129
P(scuola              |iro=0)	= 0.0114
P(risata              |iro=0)	= 0.0107


In [16]:
# sorted_hashtags = sorted(tmd_dict[0].items(), key=lambda x: x[1], reverse=True)
tot_iro = sum(tmd_dict[1].values())
p_hastg_iro = {}

for _, elem in enumerate(sorted(tmd_dict[1].items(), key=lambda x: x[1], reverse=True)):
    if (elem[1]/tot_iro) < prob_thr : break
    if _ > n_top_tweet : break

    hashtag = elem[0].ljust(20)  # Allinea l'hashtag a sinistra con 20 caratteri di spazio
    print(f'P({hashtag}|iro=1)\t= {elem[1]/tot_iro:.4f}')
    relevant_tweets.add(elem[0])
    p_hastg_iro[elem[0]] = elem[1]/tot_iro

P(monti               |iro=1)	= 0.1661
P(labuonascuola       |iro=1)	= 0.1230
P(grillo              |iro=1)	= 0.0529
P(governo             |iro=1)	= 0.0259
P(renzi               |iro=1)	= 0.0216
P(risata              |iro=1)	= 0.0151
P(manovra             |iro=1)	= 0.0140
P(serviziopubblico    |iro=1)	= 0.0129


In [17]:
'''
l è la lista dei tweet con una frequenza maggiuore di .01
'''
print(relevant_tweets)

{'labuonascuola', 'manovra', 'risata', 'piazzapulita', 'scuola', 'renzi', 'monti', 'grillo', 'serviziopubblico', 'governo', 'm5s'}


supponendo ovunque distribuzioni uniformi, questa parte mi servirà poi per fare un confronto.<br>
NB questa è una super cazzata, gli hastag non hanno tutti la stessa probabilità di apparire, vediamo se va sufficientemente bene.

In [18]:
tot_iro

927

In [19]:
n_tweet = len(tmp_df_hasth) #n. di tweet che contengono gli elem che stiamo tracciando
p_iro = tot_iro/n_tweet

p = {tweet:(tmd_dict[0][tweet] + tmd_dict[1][tweet])/n_tweet for tweet in relevant_tweets}


In [20]:
p

{'labuonascuola': 0.239770279971285,
 'manovra': 0.014596793491265853,
 'risata': 0.02129696099545346,
 'piazzapulita': 0.023689877961234746,
 'scuola': 0.0208183776022972,
 'renzi': 0.01866475233309404,
 'monti': 0.19597989949748743,
 'grillo': 0.24168461354391002,
 'serviziopubblico': 0.031825795644891124,
 'governo': 0.052165589854032066,
 'm5s': 0.02273271117492223}

P(IRONICO|HASTAG) = |IRONICI CON HASTAG| / |TWEET CON HASTAG|

In [21]:
p_iro_hastg = {tweet:(tmd_dict[1][tweet])/(tmd_dict[0][tweet] + tmd_dict[1][tweet]) for tweet in p_hastg_iro.keys()}
p_iro_hastg

{'monti': 0.18803418803418803,
 'labuonascuola': 0.11377245508982035,
 'grillo': 0.048514851485148516,
 'governo': 0.11009174311926606,
 'renzi': 0.2564102564102564,
 'risata': 0.15730337078651685,
 'manovra': 0.21311475409836064,
 'serviziopubblico': 0.09022556390977443}

P(IRONICO|HASTAG) =  P(HASTG | IRONICO) P(IRONICO) / P(HASTAG)<br>
questa cosa poi andrà rimossa

In [22]:
p_iro_hastg_2 = {tweet: (p_hastg_iro[tweet] * p_iro)/p[tweet] for tweet in p_hastg_iro.keys()}
p_iro_hastg_2

{'monti': 0.18803418803418806,
 'labuonascuola': 0.11377245508982035,
 'grillo': 0.048514851485148516,
 'governo': 0.11009174311926606,
 'renzi': 0.25641025641025644,
 'risata': 0.15730337078651685,
 'manovra': 0.21311475409836064,
 'serviziopubblico': 0.09022556390977443}

vediamo quali chiavi sono dove etc....

In [23]:
p_hastg_iro.keys()


dict_keys(['monti', 'labuonascuola', 'grillo', 'governo', 'renzi', 'risata', 'manovra', 'serviziopubblico'])

In [24]:
# chiavi non iro che non sono nel dict iro
set(p_hastg_non_iro.keys())-set(p_hastg_iro.keys())

{'m5s', 'piazzapulita', 'scuola'}

In [25]:
# chiavi iro che non sono nel dict non iro
set(p_hastg_iro.keys()) - set(p_hastg_non_iro.keys())

{'manovra', 'renzi'}

In [26]:
# chiavi comuni
common_keys = list(set(p_hastg_iro.keys()).intersection(set(p_hastg_non_iro.keys())))
print(common_keys)

['labuonascuola', 'risata', 'monti', 'grillo', 'serviziopubblico', 'governo']


probs delle chiavi in comune

In [27]:
p_iro_hastg

{'monti': 0.18803418803418803,
 'labuonascuola': 0.11377245508982035,
 'grillo': 0.048514851485148516,
 'governo': 0.11009174311926606,
 'renzi': 0.2564102564102564,
 'risata': 0.15730337078651685,
 'manovra': 0.21311475409836064,
 'serviziopubblico': 0.09022556390977443}

In [28]:
for key in common_keys:
    prob = p_iro_hastg[key]
    key = key.ljust(20)
    print(f'P(iro=1|{key})\t= {prob:.4f}')

P(iro=1|labuonascuola       )	= 0.1138
P(iro=1|risata              )	= 0.1573
P(iro=1|monti               )	= 0.1880
P(iro=1|grillo              )	= 0.0485
P(iro=1|serviziopubblico    )	= 0.0902
P(iro=1|governo             )	= 0.1101


In [29]:
tot = tot_n_iro + tot_iro
sum = 0
for key in common_keys:
    sum += p_hastg_iro[key]*tot_iro
    sum += p_hastg_non_iro[key]*tot_n_iro
print(f'percentuale di tweet del train a cui viene fatto il tratamento: {sum/tot}')

percentuale di tweet del train a cui viene fatto il tratamento: 0.41082642552122584
